In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用TabNet内置算法开始训练表格模型

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/get_started_with_tabnet.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabnet/get_started_with_tabnet.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabnet/get_started_with_tabnet.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概览

本教程演示了如何在Vertex AI平台上使用TabNet内置算法服务训练自定义表格模型。

TabNet结合了两个世界的优点：它具有可解释性（类似于简单的基于树的模型），同时又具有高性能（类似于深度神经网络）。这使其非常适用于零售商、金融和保险行业应用，如信用评分预测、欺诈检测和预测。

TabNet使用一种名为序列关注的机器学习技术，在模型的每个步骤中选择要从中推理的模型特征。这种机制使得可以解释模型如何得出预测，并帮助它学习更准确的模型。TabNet不仅胜过其他神经网络和决策树，还提供可解释的特征归因。

研究论文：[TabNet：关注可解释的表格学习](https://arxiv.org/pdf/1908.07442.pdf)

了解更多关于[Tabular Workflow for TabNet](https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/tabnet)。

### 目标

在这个笔记本中，您将学习如何运行构建的 `Vertex AI TabNet` 算法来训练自定义表格模型。

本教程使用以下 Google Cloud 机器学习服务和资源：

- `Vertex AI TabNet`
- `Vertex AI Prediction`
- `Vertex AI Training`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 获取训练数据。
- 配置 `Vertex AI TabNet` 容器的训练参数。
- 使用 CSV 数据使用 `Vertex AI Training` 训练模型。
- 将模型上传为 `Vertex AI Model` 资源。
- 部署 `Vertex AI Model` 资源到 `Vertex AI Endpoint` 资源。
- 使用部署的模型进行预测。
- 调整 `Vertex AI TabNet` 模型的超参数。

数据集

本教程使用公共云存储桶`gs://cloud-samples-data/ai-platform-unified/datasets/tabular/`中的`petfinder`数据集，该数据集来自[PetFinder.my Adoption Prediction](https://www.kaggle.com/c/petfinder-adoption-prediction)。该数据集用于预测动物被领养的速度。

### 费用

本教程使用谷歌云的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用情况生成费用估算。

安装以下软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade tensorflow --quiet
! pip3 install --upgrade google-cloud-aiplatform
! gcloud components update --quiet

仅限Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 中使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行认证。请按照以下相关说明进行操作。

1. 顶点人工智能工作台
* 无需操作，因为您已经通过身份验证。

2. 本地JupyterLab实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 请参阅如何在 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务账户授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

导入库并定义常量

In [ ]:
import os

from google.cloud import aiplatform

###初始化Vertex AI SDK用于Python

初始化Vertex AI SDK用于您的项目和相应的存储桶。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU和分配给虚拟机实例的GPU数量的容器映像。例如，要使用一个具有4个Nvidia Telsa K80 GPU分配给每个VM的GPU容器映像，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，请指定 `(None, None)` 以使用一个在CPU上运行的容器映像。

了解更多关于[您区域的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：TF 2.3之前的GPU支持版本将无法加载本教程中的自定义模型。这是一个已知问题，在TF 2.3中已修复。这是由生成在服务功能中的静态图操作所引起的。如果在您自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3的容器映像。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96 \]个

*注意：以下内容不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

DEPLOY_COMPUTE = "n1-standard-4"
print("Deploy machine type", DEPLOY_COMPUTE)

设置训练容器

接下来，您使用预构建的“Vertex AI TabNet”容器来训练模型。

In [ ]:
TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai-restricted/builtin-algorithm/tab_net_v2"

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

设置预先构建的容器以进行部署

设置预先构建的Docker容器镜像进行预测。

获取最新列表，请查看[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.5".replace(".", "-")

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

获取训练数据

首先，您需要从一个公共云存储桶中获取训练数据的副本——以CSV文件的形式，并将训练数据复制到您自己的云存储桶中。

In [ ]:
# Please note that if you use csv input, the first column is the label column.

IMPORT_FILE = "petfinder-tabular-classification-tabnet-with-header.csv"
TRAINING_DATA_PATH = f"{BUCKET_URI}/data/petfinder/train.csv"

! gsutil cp gs://cloud-samples-data/ai-platform-unified/datasets/tabular/{IMPORT_FILE} {TRAINING_DATA_PATH}

### 创建并运行 `Vertex AI TabNet` 训练作业

要训练TabNet自定义模型，您需要执行两个步骤：1）创建自定义训练作业，2）运行作业。

#### 创建自定义训练作业

使用 `CustomTrainingJob` 类创建自定义训练作业，需要以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。
- `model_serving_container_image_uri`：可以为您的模型提供预测的容器的URI——要么是预构建的

In [ ]:
DATASET_NAME = "petfinder"  # Change to your dataset name.

job = aiplatform.CustomContainerTrainingJob(
    display_name=f"{DATASET_NAME}",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

print(job)

为TabNet训练配置参数设置

以下表格显示了TabNet训练作业的参数：

| 参数 | 数据类型 | 描述 | 是否必需 |
|--|--|--|--|
| `preprocess` | 布尔参数 | 指定此参数以启用自动预处理。 | 否|
| `job_dir` | 字符串 | 保存模型输出文件的云存储目录。 | 是 |
| `input_metadata_path` | 字符串 | 训练数据集的TabNet特定元数据的GCS路径。请参阅上文了解如何创建元数据。 | 否 . |
| `training_data_path` | 字符串 | 存储训练数据的云存储模式。 | 是 |
| `validation_data_path` | 字符串 | 存储评估数据的云存储模式。 | 否 |
| `test_data_path` | 字符串 | 存储测试数据的云存储模式。 | 是 |
| `input_type` | 字符串 | “bigquery”或“csv” - 输入表格数据的类型。如果提到了csv，则第一列被视为目标。如果CSV文件有标题，也传递标志“data_has_header”。如果使用“bigquery”，可以提供训练/验证数据路径，或者提供BigQuery项目、数据集和表名，以进行预处理生成训练和验证数据集。 | 是 |
| `model_type` | 字符串 | 学习任务，如分类或回归。 | 是 |
| `split_column` | 字符串 | 用于创建训练、验证和测试拆分的列名。列的值（也称为table['split_column']）应该是“TRAIN”、“VALIDATE”或“TEST”。“TEST”是可选的。仅适用于bigquery输入。 | 否 . |
| `train_batch_size` | 整数 | 训练的批量大小。 | 否 - 默认值为1024。 |
| `eval_split` | 浮点数 | 用于评估数据集的拆分比例，如果未提供`validation_data_path`。 | 否 - 默认值为0.2 |
| `learning_rate` | 浮点数 | 训练的学习率。 | 否 - 默认值为指定优化器的默认学习率。 |
| `eval_frequency_secs` | 整数 | 评估和检查点进行的频率。默认值为600。 | 否 . |
| `num_parallel_reads` | 整数 | 用于读取输入文件的线程数。我们建议设置它等于或略低于机器的CPU数量，以获得最大性能。例如，每个GPU设置为6是一个很好的默认选择。 | 是 . |
| `optimizer` | 字符串 | 训练优化器。支持任何TF2.3 Keras优化器的小写字符串名称（'sgd'、'adam'、'ftrl'等）。请参阅[ TensorFlow文档](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)。 | 否 - 默认值为'adam'。 |
| `data_cache` | 字符串 | 选择将数据缓存在“memory”、“disk”或“no_cache”中。对于大型数据集，将数据缓存在内存中会导致内存不足错误，因此，我们建议选择“disk”。您可以在配置文件中指定磁盘大小（如下面的示例）。请确保请求一个足够大（例如TB级别）的磁盘来写入数据，适用于大型（B级别）数据集。 | 否，默认值为“memory” . |
| `bq_project` | 字符串 | BigQuery项目的名称。如果input_type=bigquery并且使用了标志–preprocessing，则此项是必需的。这是指定训练、验证和测试数据路径的替代方法。 | 否 . |
| `dataset_name` | 字符串 | BigQuery数据集的名称。如果input_type=bigquery并且使用了标志–preprocessing，则此项是必需的。这是指定训练、验证和测试数据路径的替代方法。 | 否 . |
| `table_name` | 字符串 | BigQuery表的名称。如果input_type=bigquery并且使用了标志–preprocessing，则此项是必需的。这是指定训练、验证和测试数据路径的替代方法。 | 否 . |
| `loss_function_type` | 字符串 | TabNet中有几种损失函数类型。对于回归：含有mse/mae。对于分类：含有cross_entropy/weighted_cross_entropy/focal_loss。| 否 . 如果数值为“default”，我们将用mse进行回归，用cross_entropy进行分类。|
| `deterministic_data` | 布尔参数 | 数据从表格数据中读取的确定性。默认设置为False。设置为True时，实验是确定性的。为了在大型数据集上快速训练，我们建议选择deterministic_data=False，尽管结果中存在一些随机性（在大型数据集中可以忽略）. 请注意，在分布式训练中，确定性仍然无法保证，因为map-reduce导致由于有限精度的代数操作顺序而产生随机性。然而，在实践中这是非常小的，尤其在处理大型数据集时。对于需要100%确定性的情况，除了选择deterministic_data=True，我们建议使用单个GPU进行训练（例如使用MACHINE_TYPE="n1-highmem-8"）。| 否，默认值为False . |
| `stream_inputs` | 布尔参数 | 从GCS中流式处理输入数据而不是本地下载 - 此选项建议用于快速运行时。 | 否 . |
| `large_category_dim` | 整数 | 嵌入的维度 - 如果分类列的不同类别数量大于large_category_thresh，则使用large_category_dim维度的嵌入，而不是1-D嵌入。默认值为1。我们建议增加它（例如，在大多数情况下增加到 ~5，甚至在数据集中类别数量非常大的情况下增加到 ~10），如果最重要的目标是提高准确性而不是计算效率和可解释性。 | 否 . |
| `large_category_thresh` | 整数 | 分类列基数的阈值 - 如果分类列的不同类别数量大于large_category_thresh，则使用large_category_dim维度的嵌入，而不是1-D嵌入。默认值为300。我们建议降低它（例如到 ~10），如果最重要的目标是提高准确性而不是计算效率和可解释性。 | 否 . |
| `yeo_johnson_transform` | 布尔参数 | 启用可训练的Yeo-Johnson Power变换（默认禁用）。请参阅此链接：https://www.stat.umn.edu/arc/yjpower.pdf 了解更多关于Yeo-Johnson Power变换的信息。通过我们的实现，变换参数将随着TabNet一起学习，以端到端的方式进行训练。| 否 . |
| `apply_log_transform` | 布尔参数 | 如果元数据中包含对数变换统计数据并且此标志为true，则输入要素将被对数变换。如果不使用变换，请使用false，如果使用变换则使用true（默认值）。特别对于具有偏斜的数值分布的数据集，对数变换可能非常有帮助。 | 否 . |
| `apply_quantile_transform` | 布尔参数 | 如果元数据中包含分位数统计数据并且此标志为true，则输入要素将被分位数变换。如果不使用变换，请使用false，如果使用变换则使用true（默认值）。特别对于具有偏斜的数值分布的数据集，分位数变换可能非常有帮助。目前支持BigQuery输入类型。 | 否 . |
| `replace_transformed_features` | 布尔参数 | 如果为true，则对特征应用变换后该特征将被替换。如果为false（默认选项），那么变换后的特征将被添加到要素列列表中。选择true来用变换替换特征，选择false将变换后的特征作为新列附加。 | 否 . |
| `target_column` | 字符串 | 标签列的名称。请注意，对于分类，标签需要是String或整数类型。 | 否 . |
| `prediction_raw_inputs` | 布尔参数 | 如果设置了此参数，模型服务允许我们将要素作为张量字典而不是CSV行进行传递。 | 否 . |
| `exclude_key` | 布尔参数 | 如果设置了此参数，我们将在输入/输出中排除一个键。该键对于批量预测过程中的输入和保存输出具有不可预测的顺序很有帮助。该键有助于将输出与输入匹配。 | 否 . |

了解更多关于[ 使用内置TabNet算法入门](https://cloud.google.com/ai-platform/training/docs/algorithms/tab-net-start)

In [ ]:
ALGORITHM = "tabnet"
MODEL_TYPE = "classification"
MODEL_NAME = f"{DATASET_NAME}_{ALGORITHM}_{MODEL_TYPE}"

OUTPUT_DIR = f"{BUCKET_URI}/{MODEL_NAME}"
print("Output dir: ", OUTPUT_DIR)

CMDARGS = [
    "--preprocess",
    "--data_has_header",
    f"--training_data_path={TRAINING_DATA_PATH}",
    f"--job-dir={OUTPUT_DIR}",
    f"--model_type={MODEL_TYPE}",
    "--max_steps=2000",
    "--batch_size=4096",
    "--learning_rate=0.01",
    "--prediction_raw_inputs",
    "--exclude_key",
]

### 训练TabNet模型

使用`run`方法开始训练，需要传入以下参数：

- `args`：要传递给TabNet训练容器的命令行参数。
- `replica_count`：工作器副本的数量。
- `model_display_name`：如果脚本生成一个托管`Model`，则为`Model`的显示名称。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作器副本的加速器数量。

`run`方法创建一个训练流水线来训练和创建一个`Model`对象。训练流水线完成后，`run`方法将返回`Model`对象。

In [ ]:
MODEL_DIR = OUTPUT_DIR

if TRAIN_GPU:
    model = job.run(
        model_display_name=f"{DATASET_NAME}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        sync=True,
    )
else:
    model = job.run(
        model_display_name=f"{DATASET_NAME}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

print(model.gca_resource)

删除训练任务

使用`delete()`方法来删除训练任务。

In [ ]:
job.delete()

### 部署模型

在使用模型进行预测之前，您需要将其部署到一个`Endpoint`。您可以通过在`Model`资源上调用`deploy`函数来实现这一点。这将执行两个任务：

1. 为部署`Model`资源创建一个`Endpoint`资源。
2. 将`Model`资源部署到`Endpoint`资源中。


函数接受以下参数：

- `deployed_model_display_name`：部署模型的易读名称。
- `traffic_split`：流量分配到端点的百分比，以一个或多个键/值对的字典形式指定。
   - 如果只有一个模型，则指定为**{ "0": 100 }**，其中"0"表示上传的模型，100表示100%的流量。
   - 如果端点上有现有模型，想要分配流量，则使用`model_id`来指定为**{ "0": percent, model_id: percent, ... }**，其中`model_id`是要部署到端点的现有模型的模型ID。百分比必须相加等于100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作副本的加速器数量。
- `starting_replica_count`：初始提供的计算实例数量。
- `max_replica_count`：要扩展到的最大计算实例数量。在本教程中，只有一个实例被提供。

### 流量分配

`traffic_split`参数以Python字典形式指定。您可以将多个模型实例部署到一个端点，然后设置每个实例接收的流量百分比。

您可以使用流量分配逐渐将一个新模型引入生产环境。例如，如果您已经有一个在生产环境中的模型占据了100%的流量，您可以部署一个新模型到同一个端点，将10%的流量指向它，并将原始模型的流量减少到90%。这样可以在最小化对大多数用户影响的同时监控新模型的性能。

### 计算实例扩展

您可以指定一个单一实例（节点）来处理您的在线预测请求。本教程使用单一节点，因此变量`MIN_NODES`和`MAX_NODES`都设置为`1`。

如果您想要使用多个节点来处理在线预测请求，将`MAX_NODES`设置为您希望使用的最大节点数量。Vertex AI会自动扩展用于提供预测的节点数量，直至达到您设置的最大数量。参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)了解使用多个节点进行自动扩展的成本。

### Endpoint

该方法将会阻塞，直到模型部署完成，并最终返回一个`Endpoint`对象。如果这是第一次将模型部署到端点，则可能需要额外几分钟来完成资源的提供。

In [ ]:
DEPLOYED_NAME = f"{DATASET_NAME}"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_COMPUTE.name,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

### 获取服务签名

接下来，将模型下载到本地并查询其服务签名。服务签名将采用以下形式：

（"feature_name_1"，"feature_name_2"，...）

In [ ]:
import tensorflow as tf

loaded = tf.saved_model.load(MODEL_DIR + "/model")
loaded.signatures

### 进行预测

最后，您可以使用 `predict()` 方法进行预测。每个实例都用以下字典格式指定：

    { "feature_name_1": value, "feature_name_2", value, ... }

In [ ]:
prediction = endpoint.predict(
    [
        {
            "Age": 3,
            "Breed1": "Tabby",
            "Color1": "Black",
            "Color2": "White",
            "Fee": 100,
            "FurLength": "Short",
            "Gender": "Male",
            "Health": "Healthy",
            "MaturitySize": "Small",
            "PhotoAmt": 2,
            "Sterilized": "No",
            "Type": "Cat",
            "Vaccinated": "No",
        }
    ]
)

print(prediction)

超参数调整

在成功训练模型、部署模型并调用模型进行预测之后，您可能希望优化模型训练过程中使用的超参数，以提高模型的准确性和性能。请参阅 Vertex AI 文档，了解超参数调整的概述以及如何在您的 Vertex 训练任务中使用它。

在这个示例中，以下内容运行了一个 Vertex AI 超参数调整任务，包括 4 次试验，旨在最大化验证 AUC 指标。它优化的超参数是最大步数和学习率。

### 创建试验配置

接下来，您需要构建一个 YAML 文件，其中包含超参数试验设置。

In [ ]:
config = f"""studySpec:
  metrics:
  - metricId: auc
    goal: MAXIMIZE
  parameters:
  - parameterId: max_steps
    integerValueSpec:
      minValue: 2000
      maxValue: 3000
  - parameterId: learning_rate
    doubleValueSpec:
      minValue: 0.0000001
      maxValue: 0.1
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: {TRAIN_COMPUTE}
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1
    replicaCount: 1
    diskSpec:
      bootDiskType: pd-ssd
      bootDiskSizeGb: 100
    containerSpec:
      imageUri: {TRAIN_IMAGE}
      args:
      - --preprocess 
      - --data_has_header
      - --training_data_path={TRAINING_DATA_PATH}
      - --job-dir={OUTPUT_DIR}
      - --batch_size=1028
      - --model_type={MODEL_TYPE}
      - --prediction_raw_inputs
"""

!echo $'{config}' > ./config.yaml

### 执行超参数调整试验

接下来，您可以使用命令`gcloud ai hp-tuning-jobs create`来执行超参数调整作业。

作业将异步运行。您可以使用`gcloud ai hp-tuning-jobs describe`来轮询作业的状态。

In [ ]:
MAX_TRIAL_COUNT=4
PARALLEL_TRIAL_COUNT=2

output = ! gcloud ai hp-tuning-jobs create \
  --config=config.yaml \
  --max-trial-count={MAX_TRIAL_COUNT} \
  --parallel-trial-count={PARALLEL_TRIAL_COUNT} \
  --region=$REGION \
  --display-name={DATASET_NAME}

print(output)

try:
    DESCRIBE = output[5]
    print("Describe cmd:", DESCRIBE)
except Exception as e:
    print(e)

取消超参数调整作业

接下来，使用命令`gcloud ai hp-tuning-jobs cancel`来取消超参数调整作业。

In [ ]:
try:
    DESCRIBE = output[5]
    print("Describe cmd:", DESCRIBE)

    args = DESCRIBE.split(" ")
    JOB_ID = args[7]

    ! gcloud ai hp-tuning-jobs cancel {JOB_ID} --region={REGION}
except Exception as e:
    print(e)

清理

要清理本项目中使用的所有 Google Cloud 资源，您可以 [删除为本教程使用的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

# Delete BQ table
! bq rm -f {PROJECT_ID}:{DATASET_NAME}.train

try:
    endpoint.undeploy_all()
    endpoint.delete()
    model.delete()
    model_bq.delete()
except Exception as e:
    print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI